In [0]:
%pip install --quiet airspace

In [0]:
dbutils.library.restartPython()

In [0]:
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql import Row
from functools import partial
import seg
import kayday as kd
from seg.utils import DateType
from IPython.display import Image as img
from IPython.display import display as img_display
from effodata import ACDS, golden_rules, Sifter, Equality, Joiner, Lambda
from kpi_metrics import KPI, available_metrics as  amKPI, get_metrics, AliasMetric, AliasGroupby, CustomMetric
import upc_input

import airspace
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [0]:
from datetime import datetime, timedelta
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql import Row
# Start and end dates
start_date = datetime.strptime('20240823', '%Y%m%d')
end_date = datetime.strptime('20250214', '%Y%m%d')

# List to store the dates
friday_dates = []

# Generate dates
current_date = start_date
while current_date <= end_date:
    friday_dates.append(current_date.strftime('%Y%m%d'))
    current_date += timedelta(weeks=1)

# Display the list of dates
display(friday_dates)

In [0]:

model_inputs = None
for a in friday_dates:
  if a == friday_dates[0]:
    files = spark.read.parquet(f"/Volumes/personalization_dev/atc_decisioning/raupp_atc/content_tags_test_1/input_features/created_at={a}/*.snappy.parquet")
    model_inputs = files.withColumn("created_at", f.lit(a))
  
  else:
    print(a)
    files = spark.read.parquet(f"/Volumes/personalization_dev/atc_decisioning/raupp_atc/content_tags_test_1/input_features/created_at={a}/*.snappy.parquet")
    files = files.withColumn("created_at", f.lit(a)) 


    model_inputs = model_inputs.union(files)




In [0]:
model_inputs.groupBy("EMAIL_THEME", "DEFAULT_THEME").agg(f.countDistinct("GUID")).display()

In [0]:
path = "personalization_dev.atc.official_ehhn_model_tracker"
tracker = spark.read.format("delta").table(path)
tracker = tracker.filter((f.col("start_date") == "2024-08-02") & (f.col("decisioning_path") == "bau_min_score_0"))

In [0]:
datetime.strptime('20250203', '%Y%m%d')
start_date = (datetime.today() - relativedelta(months=6)).strftime('%Y%m%d')
end_date = datetime.today().strftime('%Y%m%d')

config = {
  'start_date': start_date,
  'end_date' : end_date
}
as_mapping = airspace.email_households(config)

In [0]:
features_raw = model_inputs.withColumnRenamed("GUID", "guid").join(as_mapping, on = 'guid', how = "inner")

In [0]:
print(features_raw.count())
print(features_raw.select("guid").distinct().count())
print(features_raw.select("ehhn").distinct().count())

In [0]:
features_raw_hh = features_raw.join(tracker.select("hhgroup", "ehhn", "end_date" ), on = "ehhn",how = "inner")

In [0]:
features_raw_hh = features_raw_hh.withColumn(
    "test_group",
    f.when((f.col("hhgroup") == "test") & (f.col("end_date") == "2024-09-05"), 0)
     .when((f.col("hhgroup") == "test") & (f.col("end_date") == "2024-10-24"), 1)
     .when((f.col("hhgroup") == "test") & (f.col("end_date") == "2025-01-30"), 2)
     .otherwise(-1)
)

In [0]:
test = features_raw_hh.filter(f.col("end_date") < f.col("created_at"))

test.limit(10).display()

In [0]:
features_raw_hh.filter(f.col("end_date") < f.col("created_at")).count()

In [0]:
print(features_raw_hh.count())
print(features_raw_hh.select("guid").distinct().count())
print(features_raw_hh.select("ehhn").distinct().count())
print(tracker.select("ehhn").distinct().count())

In [0]:
features_raw_hh.limit(10).display()

In [0]:
features_raw_hh.write.mode("overwrite").saveAsTable(f"sandbox_dev.tm_learning.model_features_raw")

In [0]:
raw = spark.read.table(f"sandbox_dev.tm_learning.model_features_raw")

In [0]:
raw.groupby("hhgroup").agg(f.countDistinct("guid")).display()

In [0]:
tag_path = "abfss://atc@sa8451rauppdev.dfs.core.windows.net/content_tags_test_1/missing_tags/*/*"
tags = spark.read.csv(tag_path, header=True, inferSchema=True)
tags.limit(5).display()

In [0]:
tags.filter(f.col("EMAIL_START_DT") >= "2024-08-02").distinct().display()